<a href="https://colab.research.google.com/github/EisaacJC/MOOP/blob/main/FirstAttempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np

def buckin_n6(x):
    #This works well, so no changes, i believe
    x1, x2 = x
    term1 = 100 * np.sqrt(np.abs(x2 - 0.01*x1**2))
    term2 = 0.01 * np.abs(x1 + 10)
    return term1 + term2
def himmelblau(x):
    x1, x2 = x
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2


def gradient(f, x, h):
    #Naive implementation of gradient
    grad = np.zeros_like(x)
    for i in range(len(x)):
        xh = x.copy()
        xh[i] += h
        fxh = f(xh)
        xh[i] -= 2*h
        fxh2 = f(xh)
        grad[i] = (fxh - fxh2) / (2*h)
    return grad

def hessian(f, x, h):
    H = np.zeros((len(x), len(x)))
    for i in range(len(x)):
        for j in range(i, len(x)):
            xph = x.copy()
            xph[i] += h
            xmh = x.copy()
            xmh[i] -= h
            fxph = f(xph)
            fxmh = f(xmh)
            H[i,j] = (fxph - fxmh) / (2*h)
            H[j,i] = H[i,j]
    return H

In [43]:
lb = np.array([-15, -3])
ub = np.array([-5, 3])

In [65]:
import numpy as np
def momenta(p):
    if p=="p":
        return np.random.uniform(-0.1,0.1)
def sqp(f, x0, maxiter=1000, tol=1e-4, tol_f=1e-6):
    h=0.1
    fvals = []
    points = []
    for i in range(maxiter):
        x0 = np.clip(x0, lb, ub) #this is not necessary so delete it after
        fx = f(x0) #evaluation of the function
        dfx = gradient(f, x0,h) #calculate the numerical gradient
        """remember that h towards zero can be interpreted also as stabilization
        or in some cases being lower capable of finding other minima"""
        fvals.append(fx)#append current value of the function
        points.append(x0.copy())#ok, so copy the current solution
        Hfx = hessian(f, x0,h)#numerical evaluation of Hessian of f(x)
        #Some tricks taken from stackoverflow to be less computational expensive
        dx = -np.linalg.inv(Hfx) @ dfx
        #ok, now we're only doing \nabla{f(x)}*dr, i believe(?
        grad_proj = np.dot(dfx, dx)
        t = line_search(f, x0, dx, grad_proj)
        x0 =x0+(t * dx)
        #a kind of momenta operator but a naive implementation and also check
        #if this stochastic implementation isnt bad
        if i>0 and abs(fvals[i] - fvals[i-1]) / abs(fvals[i-1]) < tol:
            #This can take out of our local minima but also worsen the
            #objective function
            #is it that ok?¡
            if np.random.uniform()>0.5:
                x0=x0+momenta("p")+t*dx
            else:
                continue
        #asegurando que tras la perturbación se encuentre dentro del espacio
        #las soluciones tienden a irse a -4
        x0 = np.clip(x0, lb, ub)
        #solución usando fuerza bruta, restringiendola
        #al espacio de búsqueda.

        #print("Iter {}, x0 = {}, f(x0) = {}".format(i, x0, fx))
    return x0, points, fvals

def line_search(f, x0, dx, grad_proj):
    """Just some code that i took from here:
    https://stackoverflow.com/questions/52204231/implementing-backtracking-line-search-algorithm-for-unconstrained-optimization-p"""
    tol_ls = 1e-6
    a, b = 0, 1
    phi = f(x0)
    while True:
        t = (a + b) / 2
        fx = f(x0 + t * dx)
        if fx < phi - grad_proj * t + 0.5 * t**2:
            b = t
        else:
            a = t
        if abs(b - a) < tol_ls:
            return t

x0 = np.asarray([np.random.uniform(-15,-5),np.random.uniform(-3,3)])
xmin, points, vals = sqp(buckin_n6, x0)
print(xmin)
print(buckin_n6(xmin))


[-13.88454379  -3.99999905]
243.50967752701726


In [70]:
lb = np.array([-5, -5])
ub = np.array([-5, 5])
x0 = np.array([0, 0])
result = sqp(himmelblau, x0)

In [71]:
result[0]

array([-5.        , -0.34406365])